Since I started this competition only few days before the competition ended it is not perfect. 

When I saw 250 attributes I immadiately got an idea to use Triplet Network for dimensionality reduction and then use XGBoost classifier on that lower dimensional space of attributes as I've never used it before and I've heard that it does really good job. There are 1000000 entries so one probably can use very powerfull models and not be afraid of overfitting. However, I didn't have enough time to do it so I randomly selected only small part of entire dataset.

In [1]:
import os
from zipfile import ZipFile
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import random
import torch
import torch.optim as optim
from tqdm import tqdm

In [2]:
folder = './data'
file_name = os.listdir(folder)[0]

def fetch_data(folder_name=folder, file_name=file_name):
    zip_path = os.path.join(folder_name, file_name)
    data_zip = ZipFile(zip_path)
    data_zip.extractall(path=folder_name)

In [4]:
fetch_data()

In [5]:
training_data = pd.read_csv(os.path.join(folder, 'train.csv'))

In [6]:
test_data = pd.read_csv(os.path.join(folder, 'test.csv'))

In [7]:
training_data.head() #A lot of attributes It seems like they are all normalized, at the end some one hot encoded vectors.

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,...,0,1,0,0,0,0,0,0,0,1
1,1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,...,0,1,0,0,0,0,0,0,0,1
2,2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,...,0,0,0,1,1,0,0,0,0,1
3,3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,...,0,0,0,0,1,0,0,0,0,1
4,4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,...,0,1,1,0,1,0,0,1,0,1


In [8]:
training_data.info() #Some information won't be displayed by default if we have a lot of columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 287 entries, id to target
dtypes: float64(240), int64(47)
memory usage: 2.1 GB


In [9]:
training_data.isna().sum(axis=0).sum() # No null values at all

0

In [10]:
#In case that we have very long dataset with a lot of attributes we can't visually inspect it.
description = training_data.describe() 
description.drop("id", axis=1, inplace=True)

In [11]:
description.index # To know what we can search in the data frame

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [12]:
# So all attributes seems to be in the interval of [0,1]
# Generally all attributes seems to be in a good condition, nothing to do with them
print(description.loc['min'][description.loc['min'] < 0].sum())
print(description.loc['max'][description.loc['max'] > 1].sum())
#One sanity check may be to check distribution of the target attribute
print(training_data['target'].value_counts()) # data is uniformly distributed so guessing randomly we should obtain 50% accuracy

0.0
0.0
1    500485
0    499515
Name: target, dtype: int64


In [13]:
#But are they standardized? 
description.loc[['mean', 'std']] # No so we can try to standardize the attributes. However Let's firt split the data on two datasets. So that
#We will standardize Validation set with the training mean not to be biased.

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
mean,0.214334,0.460218,0.129253,0.277598,0.580710,0.416619,0.386532,0.654858,0.462256,0.258031,...,0.250096,0.137164,0.144793,0.130667,0.139210,0.199331,0.156065,0.183741,0.15468,0.500485
std,0.053320,0.101316,0.120805,0.063163,0.115338,0.058231,0.133457,0.065158,0.129439,0.119081,...,0.433068,0.344021,0.351892,0.337036,0.346166,0.399498,0.362917,0.387273,0.36160,0.500000


In [38]:
#Dividing trainig_data on validation. One way to just select 20% of training data for validation. We can do it randomly as distributions of classes are almost identical
#Preferably Ill use K-Fold-CV if I had more time

from sklearn.model_selection import train_test_split
training_set, validation_set = train_test_split(training_data, test_size=0.2, random_state=42)

training_set_label = training_set['target']
validation_set_label = validation_set['target']
training_set.drop(["id", "target"], axis=1, inplace=True)
validation_set.drop(["id", "target"], axis=1, inplace=True)

In [40]:
# As we have a lot of data random sapmling gives almost the same distribution. But if we want to make sure that it will be the same
#we can use stratified sampling
print(training_set_label.value_counts() / len(training_set_label))
print(validation_set_label.value_counts() / len(validation_set_label))

1    0.500622
0    0.499377
Name: target, dtype: float64
0    0.500065
1    0.499935
Name: target, dtype: float64


In [41]:
#Could use ScikitLearn API but why not use pandas 
training_mean = training_set.mean(axis=0) #Axis 0 means that 0th axis (rows) will be collapsed. 
training_std = training_set.std(axis=0)

In [42]:
training_set = training_set.sub(training_mean, axis='columns').divide(training_std, axis='columns')
validation_set = validation_set.sub(training_mean, axis='columns').divide(training_std, axis='columns')

In [43]:
training_set.mean(axis=0).mean(), validation_set.mean(axis=0).mean(), training_set.std(axis=0).mean(), validation_set.std(axis=0).mean()

#Since these sets differ we are not able to ideally standardize validation set using training data. 

(-4.455470690245741e-17, -0.00014155983020388869, 1.0, 0.9999599812115656)

In [47]:
#We will use triplet Networks to at first get only important parameters and try to separate probes as much as possible to make
#it easier for classifier
class ChemistryDataset(Dataset):
    def __init__(self, predictors, labels):
        self.indices = predictors.reset_index().index.values #Returns row labels for df
        self.predictors = predictors.values #we need numpt array
        self.labels = labels.values #We need numpy arrays at that moment
    def __len__(self):
        return len(self.predictors)
    
    def __getitem__(self, index):
        anchor_label = self.labels[index] # To know which class assign to positive and negative in the triplet
        anchor = torch.tensor(self.predictors[index], dtype=torch.float32)
        
        #Now we need candidates for positive, But we must Exclude currently considered index!
        positive_candidates = self.indices[self.indices != index][self.labels[self.indices != index] == anchor_label]
        positive_idx = random.choice(positive_candidates)
        positive = torch.tensor(self.predictors[positive_idx], dtype=torch.float32)
        
        negative_candidates = self.indices[self.labels != anchor_label]
        negative_idx = random.choice(negative_candidates)
        negative = torch.tensor(self.predictors[negative_idx], dtype=torch.float32)
        
        return anchor, positive, negative, torch.tensor(anchor_label)

In [50]:
set_ = ChemistryDataset(training_set[:10], training_set_label[:10])
print(set_.__getitem__(8)[3], set_.__getitem__(8)[0].shape) #seems fine
set_.__getitem__(0)[3], set_.__getitem__(0)[0].shape

tensor(0) torch.Size([285])


(tensor(1), torch.Size([285]))

In [51]:
class TripletLoss(nn.Module):
    def __init__(self, margin = 1.0, type="L2"):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.type= type
        #our objective is distance(anchor, negative) - distance(anchor, positive) > margin -> we want to separate these probes
        # moving margin to the right and multiplying by -1 we get: d(a,p) - d(a,n) + m < 0 # If this is not true then we generate Loss
    def forward(self, anchor, positive, negative):
        if self.type=="L2":
            pdist = nn.PairwiseDistance(p=2)
            #there is also cdist function
            distance_positive = pdist(anchor, positive) # by default it is Euclidean distance
            distance_negative = pdist(anchor, negative)
            losses = torch.relu(distance_positive - distance_negative + self.margin)
        elif self.type == "cosine":
            #In situations where there a many dimensions sometimes that loss may work better
            cos = nn.CosineSimilarity(dim=1, eps=1e-6)
            similarity_positive = cos(anchor, positive)
            similarity_negative = cos(anchor, negative)

            losses = torch.relu(similarity_negative - similarity_positive + self.margin) #the goal it to achieve
            #similarity(anchor,positive) - margin > similarity(anchor,negative)
               
        elif self.type == "L1":
            #this loss is taken from some YT video where guy said that IBM invented it but I couldn't find paper
            #https://www.youtube.com/watch?v=dG8le1YWUI8&t=1431s
            distance_positive = torch.unsqueeze(torch.sum(torch.abs(anchor - positive), axis=1), 0)
            distance_negative = torch.unsqueeze(torch.sum(torch.abs(anchor - negative), axis=1), 0)
            concat = torch.cat((distance_positive, distance_negative), axis=0)
            probs = torch.softmax(concat, axis=0)
            loss = torch.mean(torch.abs(probs[0] + torch.abs(1.0 - probs[1])))

            return loss
            
        return losses.mean()

In [74]:
#! Sanity Check
loss = TripletLoss(margin = 0.1)
a = torch.tensor([[1, 0], [0 ,1]], dtype=torch.float)
b = torch.tensor([[0, 0], [0 ,0]], dtype=torch.float)
c = torch.tensor([[1, 0], [-1 ,0]], dtype=torch.float)
loss.forward(a,b,c)  # First triplet generater loss 1.1 and second one 0 o it should be 0.55.

tensor(0.5500)

In [75]:
#I tried some very simple Networks and then tried to make it bigger however the smaller one does better job
class Network(nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.net = nn.Sequential(
            nn.Linear(self.input_size, self.input_size*2, bias=False),
            nn.ReLU(),
            nn.BatchNorm1d(self.input_size*2), # A lot of people claim that BN should be after ReLu. Bot originally it was before
            nn.Linear(self.input_size*2, self.input_size*4),
            nn.ReLU(), 
            #nn.BatchNorm1d(self.input_size*4), # A lot of people claim that BN should be after ReLu. Bot originally it was before
            nn.Linear(self.input_size*4, output_size),
            # nn.ReLU(),
            # nn.BatchNorm1d(self.input_size*4),
            # nn.Linear(self.input_size*4, self.input_size*4, bias=False),
            # nn.ReLU(),
            # nn.BatchNorm1d(self.input_size*4), 
            # nn.Linear(self.input_size*4, self.input_size*8, bias=False),
            # nn.ReLU(),
            # nn.BatchNorm1d(self.input_size*8), #Again some says that it is not good to have Batch
            # nn.Linear(self.input_size*8, self.input_size*8),
            # nn.ReLU(),
            # nn.Linear(self.input_size*8, self.output_size)
            
        )
        
    def forward(self,x):
        return self.net(x)
        

In [78]:
# ! Sanity Check, seems fine
net = Network(258, 10)
print(torch.randn((2,258)).dtype)
net(torch.randn((2,258))).shape

torch.float32


torch.Size([2, 10])

In [80]:
#Now the sad part as I had small amount of time and not very powerfull PC I need to cut off a lot of data :(
batch_size = 128
train_ds = ChemistryDataset(training_set[:100000], training_set_label[:100000])
train_loader = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
val_ds = ChemistryDataset(validation_set[:10000], validation_set_label[:10000])
validation_loader = DataLoader(val_ds, batch_size = batch_size, shuffle = True)

In [82]:
# Quite standard setting, additionally some easy Learning rate shceduler
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 0.001
model = Network(285, 50).to(DEVICE).float()
scaler = torch.cuda.amp.GradScaler()# This is used to scale gradients when they are to small to be represented in float16. Faster training!
loss_fn = TripletLoss(margin=0.5, type="L2")
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [83]:
#At first lets check how data is separeted without any training in validation_loader. This may make no sense but if our model perform worse
#after training on validation data than with random weights then we have a problem
loop = tqdm(validation_loader, leave=True)
val_losses = []
with torch.no_grad():
    for batch_idx, (anchor, positive, negative, targets) in enumerate(loop):
        anchor = anchor.to(DEVICE)
        positive = positive.to(DEVICE)
        negative = negative.to(DEVICE)
        
        loss = loss_fn(anchor, positive, negative)
        
        loop.set_postfix(loss=loss.item())
        val_losses.append(loss) 

print(f"Raw data score: {sum(val_losses)/len(val_losses)}")

100%|██████████| 79/79 [00:07<00:00, 10.51it/s, loss=1.17] 

Raw data score: 1.1326602697372437


In [85]:
#Training + Validation loops
#Unfortunately model overfits a little bit, using more data for sure would be beneficial.

for epoch in range(20):
    losses = []
    val_losses = []
    
    loop = tqdm(train_loader, leave=True)
    model.train()    
    for batch_idx, (anchor, positive, negative, targets) in enumerate(loop):
        #Moving data to selected device
        anchor = anchor.to(DEVICE)
        positive = positive.to(DEVICE)
        negative = negative.to(DEVICE)
        
        # This will allow doing some operations on float16 datatype. Morover this will make training faster
        with torch.cuda.amp.autocast():
            anchor_out = model(anchor)
            positive_out = model(positive)
            negative_out = model(negative)
            loss = loss_fn(anchor_out, positive_out, negative_out)
        
        #We need to use scaler for gradient to be correct, since very small gradient may be set to 0 when cast to float 16
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        loop.set_postfix(loss=loss.item())
        losses.append(loss)
        
    ## VALIDATION LOOP, almost the same stuff as for training, but we use torch_no_grad context manager and we do not calculate gradients
    loop = tqdm(validation_loader, leave=True)
    model.eval()
    with torch.no_grad():
        for batch_idx, (anchor, positive, negative, targets) in enumerate(loop):
            
            anchor = anchor.to(DEVICE)
            positive = positive.to(DEVICE)
            negative = negative.to(DEVICE)
            
            with torch.cuda.amp.autocast():
                anchor_out = model(anchor)
                positive_out = model(positive)
                negative_out = model(negative)
                loss = loss_fn(anchor_out, positive_out, negative_out)
            
            loop.set_postfix(loss=loss.item())
            val_losses.append(loss) 
    
    train_loss = sum(losses)/len(losses)
    val_loss = sum(val_losses)/len(val_losses)
    scheduler.step(val_loss)
    torch.save(model.state_dict(), f"TripletModel_{epoch}.pth") #Save model from every epoch
    print(f"Epoch: {epoch+1}, Training Loss: {train_loss}, Validation Loss: {val_loss}")
    

100%|██████████| 79/79 [00:02<00:00, 33.97it/s, loss=0.324]


Epoch: 1, Training Loss: 0.4800557494163513, Validation Loss: 0.39670270681381226


100%|██████████| 79/79 [00:02<00:00, 33.62it/s, loss=0.17] 


Epoch: 2, Training Loss: 0.3729085922241211, Validation Loss: 0.38291510939598083


100%|██████████| 79/79 [00:02<00:00, 33.49it/s, loss=0.342]


...

Epoch: 4, Training Loss: 0.32847321033477783, Validation Loss: 0.3955220580101013


100%|██████████| 79/79 [00:02<00:00, 33.49it/s, loss=0.471]


Epoch: 5, Training Loss: 0.3003995418548584, Validation Loss: 0.404936283826828


  8%|▊         | 66/782 [00:13<02:27,  4.85it/s, loss=0.236]


KeyboardInterrupt: 

In [86]:
model = Network(285, 50).to("cuda").float()
model.load_state_dict(torch.load("TripletModel_2.pth"))

<All keys matched successfully>

In [95]:
#Now we need to project our training data to the lower dimensional space with our trained model
model.eval()
model.to("cpu")
training_predictors_trans = model(torch.tensor(training_set.values[:100000,],  dtype=torch.float32)).detach().numpy()
for i in range(1,4):
    next_part = model(torch.tensor(training_set.values[100000*i:100000*(i+1),],  dtype=torch.float32)).detach().numpy()
    training_predictors_trans = np.concatenate((training_predictors_trans, next_part), axis=0)



In [96]:
#We project validation data also
validation_predictors_trans = model(torch.tensor(validation_set.values[:10000],  dtype=torch.float32)).detach().numpy()

In [97]:
#Sanity Check
print(training_predictors_trans.shape)
print(validation_predictors_trans.shape)

(400000, 50)
(10000, 50)


In [99]:
#Now lets try to learn xgboost on prepared data.
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn import  metrics   #Additional scklearn functions
from sklearn.metrics import accuracy_score

In [100]:
def modelfit(alg, predictors, labels, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    #Here we try to choose best number of estimator as this is one of the most important parameters
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(predictors, label=labels)
        
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='error', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        
        print(cvresult.shape[0])
        alg.set_params(n_estimators=cvresult.shape[0])
    
    alg.fit(predictors, labels) 
    dtrain_predictions = alg.predict(predictors)
    
    #Print model report:
    print("\nModel Report")
    print(f"Accuracy : {metrics.accuracy_score(dtrain_predictions, labels)}")  
                    
    # feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    # feat_imp.plot(kind='bar', title='Feature Importances')
    # plt.ylabel('Feature Importance Score')
        
    

In [101]:
#I've chosen most popular parameters
xgb1 = XGBClassifier(
 eval_metric="logloss",
 use_label_encoder=False, 
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

modelfit(xgb1, training_predictors_trans[:100000], training_set_label[:100000])

314

Model Report
Accuracy : 0.82539


In [103]:
#After that I tried to perform some very simple Grid Search for next 2 most important parameters however this does not lead to better results
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(
 eval_metric="logloss",
 use_label_encoder=False, 
 learning_rate =0.1,
 n_estimators=314, # I got this value from modelfit
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(training_predictors_trans[:100000], training_set_label[:100000])
print()

In [104]:
xgb1.save_model('xgboost3.model') # Lets save our model so that we can easily load it
# xgb1 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
# xgb1.load_model('xgboost3.model')

In [105]:
y_pred =xgb1.predict(validation_predictors_trans) # getting predictions

In [107]:
accuracy = accuracy_score(y_pred, validation_set_label[:10000]) # Getting accuracy score
print(accuracy) # We overfit a little bit. Almost 8% difference. Probably fitting both models with full dataset would improve this by much.

0.7513


In [109]:
test_predictors = test_data.drop(["id"], axis=1)# Mapping test predictors to numpy
test_predictors = test_predictors.sub(training_mean, axis='columns').divide(training_std, axis='columns').values 

In [110]:
model.to("cpu")
y_proba = xgb1.predict_proba(model(torch.tensor(test_predictors, dtype=torch.float32)).detach().numpy())

In [111]:
probabilities = list(y_proba[:, 1])

In [112]:
indices = list(test_data["id"])

In [113]:
with open("submission.txt", "w") as f:
    f.write("id,target\n")
    for i, score in zip(indices, probabilities):
        f.write(f"{i},{score}\n")